# Parse .xml file to produce our y

In [ ]:
import xml.etree.ElementTree as ET

# Function to parse the XML and get the note
def get_note_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Assuming the XML structure has the note under 'note' tag
    # Modify this according to your actual XML structure
    note = root.find('.//note').text  # Adjust the XPath based on your XML structure
    return note

# Function to encode the note to a number
def encode_note(note):
    note_mapping = {
        'C5': 0, 'D5': 1, 'E5': 2, 'F5': 3, 'G5': 4, 'A6': 5, 'B6': 6,
        'B5': -1, 'A5': -2, 'G4': -3, 'F4': -4, 'E4': -5,
        'D4': -6, 'C4': -7, 'B4': -8
    }
    return note_mapping.get(note, -1)  # Return -1 if note is not in mapping

# Function to process the XML and get the encoded note
def process_xml(xml_path):
    note = get_note_from_xml(xml_path)
    encoded_note = encode_note(note)
    return encoded_note

# Example usage
xml_path = 'path/to/note.xml'
encoded_note = process_xml(xml_path)
print(f"Encoded Note: {encoded_note}")


# Crop images to produce X

In [ ]:
from PIL import Image

def crop_note_from_png(image_path, output_path):
    """
    Crops the image to the specified dimensions.

    Parameters:
    - image_path (str): The path to the input PNG image.
    - output_path (str): The path to save the cropped image.
    """
    # Define the crop box (left, upper, right, lower)

    crop_box = (506, 536, 580, 870)  # Replace these values with your desired dimensions

    # Open the image file
    with Image.open(image_path) as img:
        # Crop the image using the provided crop box
        cropped_img = img.crop(crop_box)

        # Save the cropped image
        cropped_img.save(output_path)

# Example usage
image_path = 'path/to/input_image.png'
output_path = 'path/to/output_image.png'
crop_note_from_png(image_path, output_path)


# Cropper but every .png in a directory

In [2]:
import os
from PIL import Image

def crop_note_from_png_folder(input_folder, output_folder):
    """
    Crops all PNG images in the specified folder to the specified dimensions.

    Parameters:
    - input_folder (str): The path to the input folder containing PNG images.
    - output_folder (str): The path to the folder to save the cropped images.
    """
    # Define the crop box (left, upper, right, lower)
    crop_box = (506, 536, 580, 870)  # Replace these values with your desired dimensions

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith('.png'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Open the image file
            with Image.open(input_path) as img:
                # Crop the image using the provided crop box
                cropped_img = img.crop(crop_box)

                # Save the cropped image
                cropped_img.save(output_path)

            print(f'Cropped image saved to {output_path}')

# Example usage
input_folder = 'raw_data/sheet_images'
output_folder = 'raw_data/cropped_images'
crop_note_from_png_folder(input_folder, output_folder)


FileNotFoundError: [Errno 2] No such file or directory: 'raw_data/sheet_images'

# Cropper for lines 2-6 of sheet music

In [ ]:
import os
from PIL import Image

def crop_multiple_notes_from_png_folder(input_folder, output_folder, crop_boxes):
    """
    Crops specified regions from PNG images in the given folder based on the provided crop boxes.

    Parameters:
    - input_folder (str): The path to the input folder containing PNG images.
    - output_folder (str): The path to the folder to save the cropped images.
    - crop_boxes (list of tuples): A list of tuples, where each tuple contains the crop box dimensions (left, upper, right, lower).
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith('.png'):
            input_path = os.path.join(input_folder, filename)

            # Open the image file
            with Image.open(input_path) as img:
                # Iterate through the crop boxes
                for i, crop_box in enumerate(crop_boxes):
                    # Crop the image using the current crop box
                    cropped_img = img.crop(crop_box)

                    # Define the output path for the cropped image
                    output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_line_{i+1}.png")

                    # Save the cropped image
                    cropped_img.save(output_path)

                    print(f'Cropped image {i+1} saved to {output_path}')

# Example usage
input_folder = 'path/to/input_folder'
output_folder = 'path/to/output_folder'

# Define the crop boxes (left, upper, right, lower) for the 5 different lines of music
crop_boxes = [
    (316, 1016, 2762, 1380),  # First line
    (316, 1532, 2762, 1896),  # Second line
    (316, 2048, 2762, 2412),  # Third line
    (316, 2562, 2762, 2926),  # Fourth line
    (316, 3078, 2762, 3442)   # Fifth line
]

crop_multiple_notes_from_png_folder(input_folder, output_folder, crop_boxes)


## The code below maps our range of notes in a measure, starting from 0 at the bottom and 16 at the top (can be adjusted ofc) 

We will make a mapping dictionary that takes the location of the note and the duration and turns it into an int. For example, 0 == (0, 'whole'), 1 == (1, 'whole') etc. The reverse mapping dict will simply allow us to take that interger and turn it back into a tuple. 

In [ ]:
import numpy as np

# Define the positions (0-16) and durations (whole, half, quarter, eighth, sixteenth)
positions = np.arange(17)  # 0-16
durations = ["whole", "half", "quarter", "eighth", "sixteenth"]

# Create a mapping dictionary
mapping_dict = {}
reverse_mapping_dict = {}

label_counter = 0

for position in positions:
    for duration in durations:
        # Create a unique label for each position-duration combination
        mapping_dict[(position, duration)] = label_counter
        reverse_mapping_dict[label_counter] = (position, duration)
        label_counter += 1

# Print the mapping to see how it looks
print("Mapping Dictionary (Position, Duration -> Label):")
print(mapping_dict)
print("\nReverse Mapping Dictionary (Label -> Position, Duration):")
print(reverse_mapping_dict)


## More complex version below, with common key signatures and flat/sharp 'accidental' recognition

Will create tuple with (position, duration, accidental, key)

In [ ]:
# Define the positions (0-16) and durations, plus accidentals
positions = np.arange(17)  # 0-16 for A4 to C7
durations = ["whole", "half", "quarter", "eighth", "sixteenth"]
accidentals = [0, 1, -1]  # 0 for natural, 1 for sharp, -1 for flat

# Define the 12 common key signatures
key_signatures = [
    "C Major", "G Major", "D Major", "A Major",
    "F Major", "Bb Major", "Eb Major",
    "A Minor", "E Minor", "B Minor", "D Minor", "G Minor"
]

# Create a mapping dictionary with key signatures included
mapping_dict = {}
reverse_mapping_dict = {}

label_counter = 0

for position in positions:
    for duration in durations:
        for accidental in accidentals:
            for key in key_signatures:
                # Create a unique label for each position-duration-accidental-key combination
                mapping_dict[(position, duration, accidental, key)] = label_counter
                reverse_mapping_dict[label_counter] = (position, duration, accidental, key)
                label_counter += 1

# Print the mapping to see how it looks
print("Mapping Dictionary (Position, Duration, Accidental, Key -> Label):")
print(mapping_dict)
print("\nReverse Mapping Dictionary (Label -> Position, Duration, Accidental, Key):")
print(reverse_mapping_dict)


## Example mapping for MIDI with based on position in C Major


In [ ]:
position_to_midi = {
    0: 69,   # A4
    1: 71,   # B4
    2: 72,   # C5
    3: 74,   # D5
    4: 76,   # E5
    5: 77,   # F5
    6: 79,   # G5
    7: 81,   # A5
    8: 83,   # B5
    9: 84,   # C6
    10: 86,  # D6
    11: 88,  # E6
    12: 89,  # F6
    13: 91,  # G6
    14: 93,  # A6
    15: 95,  # B6
    16: 96   # C7
}


## Adjusting notes for key signature

In [ ]:
def adjust_for_key(midi_note, key):
    key_adjustments = {
        "G Major": {77: 78},  # F5 -> F#5
        "D Major": {77: 78, 79: 80},  # F5 -> F#5, C6 -> C#6
        "A Major": {77: 78, 79: 80, 84: 85},  # F5 -> F#5, C6 -> C#6, G6 -> G#6
        "F Major": {71: 70},  # B4 -> Bb4
        "Bb Major": {71: 70, 76: 75},  # B4 -> Bb4, E5 -> Eb5
        "Eb Major": {71: 70, 76: 75, 81: 80},  # B4 -> Bb4, E5 -> Eb5, A5 -> Ab5
        "A Minor": {},  # A Minor has no accidental adjustments
        "E Minor": {77: 78},  # F5 -> F#5
        "B Minor": {77: 78, 79: 80},  # F5 -> F#5, C6 -> C#6
        "D Minor": {71: 70},  # B4 -> Bb4
        "G Minor": {71: 70, 76: 75},  # B4 -> Bb4, E5 -> Eb5
        "C Major": {}  # C Major has no accidental adjustments
    }
    if midi_note in key_adjustments.get(key, {}):
        return key_adjustments[key][midi_note]
    return midi_note


## Creating MIDI with just position and duration.

In [ ]:
from mido import MidiFile, MidiTrack, Message

# Function to create the MIDI file
def create_midi(notes, output_file='output.mid', ticks_per_beat=480):
    mid = MidiFile(ticks_per_beat=ticks_per_beat)
    track = MidiTrack()
    mid.tracks.append(track)

    # Set a default instrument (optional)
    track.append(Message('program_change', program=0, time=0))

    for pitch, duration in notes:
        track.append(Message('note_on', note=pitch, velocity=64, time=0))
        track.append(Message('note_off', note=pitch, velocity=64, time=duration))

    mid.save(output_file)

# Assuming reverse_mapping_dict exists and looks something like:
# reverse_mapping_dict = {0: (0, 'quarter'), 1: (1, 'half'), ...}

# Map position to MIDI pitch (starting from C4, MIDI 60)
position_to_midi = {i: 60 + i for i in range(18)}  # Adjust as needed

# Map durations to ticks
duration_mapping = {
    'whole': 1920,    # 4 beats
    'half': 960,      # 2 beats
    'quarter': 480,   # 1 beat
    'eighth': 240,    # 0.5 beat
    'sixteenth': 120  # 0.25 beat
}

# Create the notes list using reverse_mapping_dict
notes = []
for label, (position, duration_str) in reverse_mapping_dict.items():
    pitch = position_to_midi[position]  # Get MIDI pitch from position
    duration = duration_mapping[duration_str]  # Get ticks from duration
    notes.append((pitch, duration))

# Now, use the notes list to create the MIDI file
create_midi(notes, output_file='my_music.mid')


## Tracking key signature and accidentals also.

In [ ]:
from mido import MidiFile, MidiTrack, Message
import numpy as np

# Function to create the MIDI file
def create_midi(notes, output_file='output.mid', ticks_per_beat=480):
    mid = MidiFile(ticks_per_beat=ticks_per_beat)
    track = MidiTrack()
    mid.tracks.append(track)

    # Set a default instrument (optional)
    track.append(Message('program_change', program=0, time=0))

    for pitch, duration in notes:
        track.append(Message('note_on', note=pitch, velocity=64, time=0))
        track.append(Message('note_off', note=pitch, velocity=64, time=duration))

    mid.save(output_file)

# Assuming reverse_mapping_dict exists and contains mappings as described
# Map positions to MIDI note numbers, assuming C4 (MIDI 60) as a base and accidentals
position_to_midi = {i: 60 + i for i in range(17)}  # A4 to C7
duration_mapping = {
    'whole': 1920,    # 4 beats
    'half': 960,      # 2 beats
    'quarter': 480,   # 1 beat
    'eighth': 240,    # 0.5 beat
    'sixteenth': 120  # 0.25 beat
}

# Create the notes list using reverse_mapping_dict
notes = []
for label, (position, duration_str, accidental, key) in reverse_mapping_dict.items():
    pitch = position_to_midi[position] + accidental  # Adjust pitch for accidentals
    duration = duration_mapping[duration_str]  # Get duration in ticks
    notes.append((pitch, duration))

# Now, use the notes list to create the MIDI file
create_midi(notes, output_file='my_music.mid')
